In [ ]:
from rerankers import Reranker
from rich import print

In [ ]:
# ranker = Reranker('cross-encoder')
# ranker = Reranker('avsolatorio/GIST-large-Embedding-v0')
# ranker = Reranker('flashrank')
# ranker = Reranker('ce-esci-MiniLM-L12-v2', model_type='flashrank')
# ranker = Reranker("t5")
# ranker = Reranker("unicamp-dl/InRanker-base", model_type = "t5")
# ranker = Reranker("colbert")
ranker = Reranker(
    "mixedbread-ai/mxbai-rerank-large-v1", model_type="cross-encoder"
)

In [ ]:
texts = [
    "I like to play soccer",
    "I like to play football",
    "I like to play basketball",
    "I love dogs",
    "Catcher in the Rye is a great book",
]

In [ ]:
results = ranker.rank(query="What's your favorite sport?", docs=texts)

print(results)

In [ ]:
print([document.text for document in results.results])

# Testing `query_similar_docs`

In [ ]:
embedded_question = get_embeddings(question)
db_conn = get_db_conn()
num_docs = 20 if use_reranking else 5
# get (content, url) tuples for the top n similar documents
top_similar_docs = get_topn_similar_docs(
    embedded_question, db_conn, n=num_docs, include_metadata=True
)

if use_reranking:
    urls = rerank_documents(question, top_similar_docs)[:5]
else:
    urls = [doc[1] for doc in top_similar_docs]  # Unpacking URLs

return (question, url_ending, urls)

# Simplistic TF-IDF Reranker

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Toy document corpus
documents = [
    "The quick brown fox jumps over the lazy dog",
    "A quick brown fox jumps over the lazy dog",
    "The quick brown fox is quick and brown",
    "The lazy dog is lazy and sleepy",
    "The quick brown fox is different from the lazy dog",
]

# Toy queries and their corresponding relevant document indices
queries = [
    ("quick fox", [0, 1, 2]),
    ("lazy dog", [3, 4]),
    ("brown fox", [0, 1, 2, 4]),
]

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Vectorize documents
document_vectors = vectorizer.fit_transform(documents)

# Train a logistic regression reranker
reranker = LogisticRegression(random_state=42)
X_train = []
y_train = []

for query, relevant_docs in queries:
    query_vector = vectorizer.transform([query])
    for doc_idx, doc_vector in enumerate(document_vectors):
        X_train.append(
            np.concatenate(
                (query_vector.toarray()[0], doc_vector.toarray()[0])
            )
        )
        y_train.append(1 if doc_idx in relevant_docs else 0)

reranker.fit(X_train, y_train)

# Example usage of the trained reranker
query = "quick brown fox"
query_vector = vectorizer.transform([query])
scores = []

for doc_vector in document_vectors:
    input_vector = np.concatenate(
        (query_vector.toarray()[0], doc_vector.toarray()[0])
    )
    score = reranker.predict_proba([input_vector])[0][1]
    scores.append(score)

ranked_indices = np.argsort(scores)[::-1]
print("Ranked documents for query:", query)
for idx in ranked_indices:
    print("Document:", documents[idx])
    print("Score:", scores[idx])
    print()

In [ ]:
from rerankers import Reranker

ranker = Reranker("cross-encoder")

texts = [
    "I like to play soccer",
    "I like to play football",
    "War and Peace is a great book",
    "I love dogs",
    "Ginger cats aren't very smart",
    "I like to play basketball",
]

results = ranker.rank(query="What's your favorite sport?", docs=texts)

In [ ]:
print(results)